In [1]:
import requests
import json
# from langchain_openai import ChatOpenAI
import os
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate
import os
import pathlib
from bs4 import BeautifulSoup
import re
from bs4 import BeautifulSoup
import re
from langchain_openai import ChatOpenAI
from pydantic import Field
from typing import Literal
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage, AIMessage
from __future__ import print_function
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from IPython.display import Markdown, display
from typing import Optional

llm_model = os.environ["LLM_VERSION"]

search_starters = ["Digital Marketing Agencies", "Renewable Energy Companies", "Cybersecurity Firms", "Legal Services Providers", "Healthcare Consultancies"]

class ResultRelevance(BaseModel):
    explanation: str
    link: str

class RelevanceCheckOutput(BaseModel):
    relevant_results: List[ResultRelevance]

class SqlCommand(BaseModel):
    command: str

class ServiceProviderMemberDetails(BaseModel):
    name: Optional[str]
    role_description: Optional[str]
    telephone: Optional[str]
    mobile: Optional[str]
    email: Optional[str]
    linkedin: Optional[str]
    facebook: Optional[str]
    instagram: Optional[str]
    twitter: Optional[str]
    additional_info: Optional[str]

class ServiceProviderOutput(BaseModel):
    service_description: Optional[str]
    rating: Optional[str]
    pricing: Optional[str]
    service_tags: Optional[List[str]]
    provider_type: Optional[str] = "Company"
    country: Optional[str]
    name: Optional[str]
    logo: Optional[str]
    website: Optional[str]
    linkedin: Optional[str]
    facebook: Optional[str]
    instagram: Optional[str]
    telephone: Optional[str]
    mobile: Optional[str]
    emails: Optional[List[str]]
    office_locations: Optional[str]
    key_individuals: Optional[str]
    service_provider_member_details: Optional[List[ServiceProviderMemberDetails]]

class SearchTermsOutput(BaseModel):
    search_terms: List[str]

class AboutUsOutput(BaseModel):
    about_us_link: Optional[List[str]]

def search_serper(search_query):
    url = "https://google.serper.dev/search"
    
    payload = json.dumps({
        "q": search_query,
        "gl": "ae", 
        "num": 10,
        "tbs": "qdr:d"
    })

    headers = {
        'X-API-KEY': os.environ["SERPER_API_KEY"],
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    results = json.loads(response.text)
    results_list = results['organic']

    all_results = []
    for id, result in enumerate(results_list, 1):
        result_dict = {
            'title': result['title'],
            'link': result['link'],
            'snippet': result['snippet'],
            'search_term': search_query,
            'id': id
        }
        all_results.append(result_dict)
    return all_results


# def load_prompt(prompt_name, html_content=None, sql_schema=None):
#     if prompt_name == "generate_sql":
#         if not html_content or not sql_schema:
#             raise ValueError("html_content and sql_schema must be provided for 'generate_sql' prompt.")

#         prompt = f"""
# You are given HTML webpage content. Extract the details precisely:

# - Business name, description, category, country of origin.
# - Each office location city and country.
# - Staff member details: name, position, email, phone, and social media accounts.

# Assume the PostgreSQL tables (`businesses`, `office_locations`, `staff`, `staff_social_media`) are already created with the following schema:

# {sql_schema}

# Do NOT include any CREATE TABLE commands. Assume `business_id` and `staff_id` are SERIAL PRIMARY KEYS, retrieved using RETURNING clauses in PostgreSQL.

# Provide exactly ONE executable PostgreSQL transaction (wrapped in BEGIN; ... COMMIT;) containing only valid INSERT commands, precisely formatted and ready to run through a Python script executing SQL commands. No additional explanations or formatting outside the SQL code.

# HTML Content:
# {html_content}
# """
#         return prompt

#     else:
#         raise ValueError(f"Unsupported prompt_name '{prompt_name}'")

def load_prompt(prompt_name):
    with open(f"prompts/{prompt_name}.md", "r") as f:
        return f.read()



def check_search_relevance(search_term: str, search_results: Dict[str, Any]) -> RelevanceCheckOutput:
    """
    Analyze search results and determine the most relevant ones.
    
    Args:
        search_results: Dictionary containing search results to analyze
        
    Returns:
        RelevanceCheckOutput containing the most relevant results and explanation
    """
    prompt = load_prompt("relevance_check")
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", prompt)
    ])

    llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model, temperature=0.1).with_structured_output(RelevanceCheckOutput)
    llm_chain = prompt_template | llm
    
    return llm_chain.invoke({"search_term": search_term, 'search_results': search_results})


def convert_html_to_markdown(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Headers
    for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        level = int(h.name[1])
        h.replace_with('#' * level + ' ' + h.get_text() + '\n\n')
    
    # Links
    for a in soup.find_all('a'):
        href = a.get('href', '')
        text = a.get_text()
        if href and text:
            a.replace_with(f'[{text}]({href})')
    
    # Bold
    for b in soup.find_all(['b', 'strong']):
        b.replace_with(f'**{b.get_text()}**')
    
    # Italic
    for i in soup.find_all(['i', 'em']):
        i.replace_with(f'*{i.get_text()}*')
    
    # Lists
    for ul in soup.find_all('ul'):
        for li in ul.find_all('li'):
            li.replace_with(f'- {li.get_text()}\n')
    
    for ol in soup.find_all('ol'):
        for i, li in enumerate(ol.find_all('li'), 1):
            li.replace_with(f'{i}. {li.get_text()}\n')
    
    # Get text and clean up
    text = soup.get_text()
    
    # Remove excess whitespace/newlines
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = text.strip()
    
    return text

def scrape_and_save_markdown(relevant_results):
    """
    Scrapes HTML content from URLs in relevant_results and saves as markdown files.
    
    Args:
        relevant_results: List of dictionaries containing search results with URLs
        
    Returns:
        List of dictionaries containing markdown content and metadata
    """
    # Create scraped_html directory if it doesn't exist
    # pathlib.Path("scraped_markdown").mkdir(exist_ok=True)

    markdown_contents = []
    for result in relevant_results:
        if 'link' in result:
            payload = {
                "api_key": os.environ["SCRAPING_API_KEY"], 
                "url": result['link'],
                "render_js": "true"
            }

            response = requests.get("https://scraping.narf.ai/api/v1/", params=payload)
            if response.status_code == 200:
                # Create filename from ID or URL if ID not available
                # filename = f"{result.get('id', hash(result['link']))}.md"
                # filepath = os.path.join("scraped_markdown", filename)
                
                # Convert HTML to markdown
                markdown_content = convert_html_to_markdown(response.content.decode())
                
                # Save markdown content to file
                # with open(filepath, 'w', encoding='utf-8') as f:
                #     f.write(markdown_content)
                
                markdown_contents.append({
                    'url': result['link'],
                    # 'filepath': filepath,
                    'markdown': markdown_content,
                    'title': result.get('title', ''),
                    'id': result.get('id', '')
                })
            else:
                print(f"Failed to fetch {result['link']}: Status code {response.status_code}")

    # print(f"Successfully downloaded and saved {len(markdown_contents)} pages as markdown to scraped_markdown/")
    print(f"Successfully downloaded and saved {len(markdown_contents)} pages as markdown")
    return markdown_contents

In [2]:
prompt = load_prompt("generate_search_terms")
service = search_starters[3]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt)
])

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(SearchTermsOutput)
llm_chain = prompt_template | llm

result = llm_chain.invoke({"service": service})

print("prompt:\n")
print(prompt)
print("\n")
print("result:\n")
print(result)
print("\n")


/home/mohammed/anaconda3/envs/growbal/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


prompt:

# Role

You are an expert at generating concise, targeted Google search terms specifically intended to discover official websites of companies or service providers within a given industry or those providing specific services.

# Task

Your generated search terms should effectively yield official company websites when entered into Google.

Provide exactly a JSON object containing the extracted fields.

Provided industry or service:
{service}

# Output

A JSON object containing the extracted fields.



result:

search_terms=['Legal services providers official website', 'Law firms official website', 'Legal consultancies official website', 'Top legal service companies website', 'Legal advisory firms official site']




In [3]:
# links = []
for search_term in result.search_terms:
    links = search_serper(search_term)
    relevant_links = check_search_relevance(search_term, links)
    # print(relevant_links)
    break

# print(links)

/home/mohammed/anaconda3/envs/growbal/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [ ]:
import os
import django
import sys
from asgiref.sync import sync_to_async

sys.path.insert(0, '/home/mohammed/Desktop/tech_projects/growbal/growbal_django')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "growbal.settings")

django.setup()

from accounts.models import CustomUser
from services.serializers import ServiceSerializer
from accounts.serializers import ServiceProviderProfileSerializer
from asgiref.sync import sync_to_async


@sync_to_async
def get_or_create_user(email, username, password):
    user, created = CustomUser.objects.get_or_create(
        email=email,
        defaults={'username': username}
    )
    if created and password:
        user.set_password(password)
        user.save()
    return user.id

@sync_to_async
def create_service(data):
    serializer = ServiceSerializer(data=data)
    if serializer.is_valid():
        service = serializer.save()
        return {
            "id": service.id,
            "description": service.service_description,
            "tags": list(service.service_tags.names())
        }
    else:
        # raise ValueError(serializer.errors)
        print(serializer.errors)

@sync_to_async
def create_service_provider_profile(data):
    serializer = ServiceProviderProfileSerializer(data=data)
    if serializer.is_valid():
        profile = serializer.save()
        return {
            "username": profile.user.username,
            "profile_id": profile.id,
            "name": profile.name,
        }
    else:
        print(serializer.errors)
        # raise ValueError(serializer.errors)

In [ ]:
from scrapper import HtmlScraper
import tiktoken
import string
import secrets

def generate_password(length=8):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for _ in range(length))
    return password

def count_tokens(text, model="gpt-4"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

scraper = HtmlScraper(scraping_api_key=os.environ["SCRAPING_API_KEY"])
for relevant_link in relevant_links.relevant_results:
    try:
        html_content = scraper.scrape_html(relevant_link.link)
        nav_links = scraper.get_nav_links(html_content)
        nav_links_str = "\n".join([f"{text}: {link}" for text, link in nav_links])

        prompt = load_prompt("generate_about_us_link")
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", prompt)
        ])
        llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(AboutUsOutput)
        llm_chain = prompt_template | llm

        result = llm_chain.invoke({"nav_links": nav_links_str, "base_url": relevant_link.link})
        print(result)
        print(result.about_us_link)
        print("\n")
        clean_html_sum = ""
        i = 1
        num_tokens_allowed = 9000
        for link in result.about_us_link:
            if count_tokens(clean_html_sum) >= num_tokens_allowed:
                continue
            try:
                html_content = scraper.scrape_html(link)
                clean_text, clean_html = scraper.clean_html_content(html_content)
                print(count_tokens(clean_html))
                if count_tokens(clean_html) < 20000/len(result.about_us_link):
                    clean_html_sum += f"HTML PAGE {i}: ({link})\n\n"
                    clean_html_sum += clean_html + "\n\n"
                    print("approved")
                i += 1
            except Exception as e:
                print(f"Error processing {link}: {e}")
        
        clean_html_sum = clean_html_sum[:num_tokens_allowed]
        if clean_html_sum != "":
            prompt = load_prompt("generate_fields")
            prompt_template = ChatPromptTemplate.from_messages([
                ("system", prompt)
            ])
            llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(ServiceProviderOutput)
            llm_chain = prompt_template | llm
            result = llm_chain.invoke({"html_content": clean_html_sum})
            # print(result)
            print(result)
        # print(nav_links)

        if hasattr(result, 'name') and result.name and result.name != "":
            from growbal_django.accounts.management.create_or_get_user import get_or_create_user

            # Now use the function directly
            if hasattr(result, 'emails') and result.emails and result.emails[0] != "":
                user_email = result.emails[0]
                user_username = user_email
                user_password = generate_password(8)
                user_id = get_or_create_user(email=user_email, username=user_username, password=user_password)
            elif result.name:
                user_email = f"contact@{result.name}.com"
                user_username = user_email
                user_password = generate_password(8)
                user_id = get_or_create_user(email=user_email, username=user_username, password=user_password)
            else:
                continue
            # print(f"User ID: {user_id}")
            service_data_json = {
                "service_description": "Professional Legal Consulting",
                "service_tags": ["legal", "consulting", "business"],
                "ratings": "4.7/5",
                "pricing": "Hourly rate starting at $150"
            }

            # Direct async call (works in Jupyter/IPython):
            service_saved = await create_service(service_data_json)
            print(f"Service created: {service_saved}")
            data = {
                    "user": user_email,
                    "service": service_saved["id"],  # existing Service ID or None
                    "provider_type": result.provider_type,
                    "country": result.country,
                    "session_status": "inactive",
                    "tier": "Level 1",
                    "name": result.name,
                    "website": result.website,
                    "linkedin": result.linkedin,
                    "facebook": result.facebook,
                    "instagram": result.instagram,
                    "telephone": result.telephone,
                    "mobile": result.mobile,
                    "emails": result.emails,
                    "office_locations": result.office_locations,
                    "key_individuals": result.key_individuals
                }
            saved_profile = await create_service_provider_profile(data)
            print(f"Profile created for user: {saved_profile['username']}")
            print(f"Profile ID: {saved_profile['profile_id']}")
            print(f"Company Name: {saved_profile['name']}")
    except Exception as e:
        print(f"Error processing {relevant_link.link}: {e}")
        continue

In [ ]:
from scrapper import HtmlScraper
import tiktoken
import string
import secrets

def generate_password(length=8):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for _ in range(length))
    return password

def count_tokens(text, model="gpt-4"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

scraper = HtmlScraper(scraping_api_key=os.environ["SCRAPING_API_KEY"])
for relevant_link in relevant_links.relevant_results:
    try:
        html_content = scraper.scrape_html(relevant_link.link)
        nav_links = scraper.get_nav_links(html_content)
        nav_links_str = "\n".join([f"{text}: {link}" for text, link in nav_links])

        prompt = load_prompt("generate_about_us_link")
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", prompt)
        ])
        llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(AboutUsOutput)
        llm_chain = prompt_template | llm

        result = llm_chain.invoke({"nav_links": nav_links_str, "base_url": relevant_link.link})
        print(result)
        print(result.about_us_link)
        print("\n")
        clean_html_sum = ""
        i = 1
        num_tokens_allowed = 9000
        for link in result.about_us_link:
            if count_tokens(clean_html_sum) >= num_tokens_allowed:
                continue
            try:
                html_content = scraper.scrape_html(link)
                clean_text, clean_html = scraper.clean_html_content(html_content)
                print(count_tokens(clean_html))
                if count_tokens(clean_html) < 20000/len(result.about_us_link):
                    clean_html_sum += f"HTML PAGE {i}: ({link})\n\n"
                    clean_html_sum += clean_html + "\n\n"
                    print("approved")
                i += 1
            except Exception as e:
                print(f"Error processing {link}: {e}")
        
        clean_html_sum = clean_html_sum[:num_tokens_allowed]
        if clean_html_sum != "":
            prompt = load_prompt("generate_fields")
            prompt_template = ChatPromptTemplate.from_messages([
                ("system", prompt)
            ])
            llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(ServiceProviderOutput)
            llm_chain = prompt_template | llm
            result = llm_chain.invoke({"html_content": clean_html_sum})
            # print(result)
            print(result)
        # print(nav_links)

        if hasattr(result, 'name') and result.name and result.name != "":
            from growbal_django.accounts.management.create_or_get_user import get_or_create_user

            # Now use the function directly
            if hasattr(result, 'emails') and result.emails and result.emails[0] != "":
                user_email = result.emails[0]
                user_username = user_email
                user_password = generate_password(8)
                user_id = get_or_create_user(email=user_email, username=user_username, password=user_password)
            elif result.name:
                user_email = f"contact@{result.name}.com"
                user_username = user_email
                user_password = generate_password(8)
                user_id = get_or_create_user(email=user_email, username=user_username, password=user_password)
            else:
                continue
            # print(f"User ID: {user_id}")
            service_data_json = {
                "service_description": "Professional Legal Consulting",
                "service_tags": ["legal", "consulting", "business"],
                "ratings": "4.7/5",
                "pricing": "Hourly rate starting at $150"
            }

            # Direct async call (works in Jupyter/IPython):
            service_saved = await create_service(service_data_json)
            print(f"Service created: {service_saved}")
            data = {
                    "user": user_email,
                    "service": service_saved["id"],  # existing Service ID or None
                    "provider_type": result.provider_type,
                    "country": result.country,
                    "session_status": "inactive",
                    "tier": "Level 1",
                    "name": result.name,
                    "website": result.website,
                    "linkedin": result.linkedin,
                    "facebook": result.facebook,
                    "instagram": result.instagram,
                    "telephone": result.telephone,
                    "mobile": result.mobile,
                    "emails": result.emails,
                    "office_locations": result.office_locations,
                    "key_individuals": result.key_individuals
                }
            saved_profile = await create_service_provider_profile(data)
            print(f"Profile created for user: {saved_profile['username']}")
            print(f"Profile ID: {saved_profile['profile_id']}")
            print(f"Company Name: {saved_profile['name']}")
    except Exception as e:
        print(f"Error processing {relevant_link.link}: {e}")
        continue

/home/mohammed/anaconda3/envs/growbal/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


about_us_link=['https://www.hilldickinson.com/about-us', 'https://www.hilldickinson.com/sectors/professional-services', 'https://www.hilldickinson.com/services/banking-and-financial-law', 'https://www.hilldickinson.com/services/commercial-law', 'https://www.hilldickinson.com/services/corporate', 'https://www.hilldickinson.com/services/dispute-resolution', 'https://www.hilldickinson.com/services/employment-law', 'https://www.hilldickinson.com/services/family', 'https://www.hilldickinson.com/services/healthcare-law', 'https://www.hilldickinson.com/marine-casualty-law-services', 'https://www.hilldickinson.com/services/notary-public', 'https://www.hilldickinson.com/services/pensions', 'https://www.hilldickinson.com/services/personal-injury', 'https://www.hilldickinson.com/services/private-client-lawyers', 'https://www.hilldickinson.com/services/property-and-construction-law', 'https://www.hilldickinson.com/services/regulatory-law', 'https://www.hilldickinson.com/services/restructuring-and-

/home/mohammed/anaconda3/envs/growbal/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


service_description='Hill Dickinson is a leading commercial law firm with over 1000 people, including over 200 partners, and boasts operations in various locations including Birmingham, Hong Kong, Leeds, Limassol, Liverpool, London, Manchester, Monaco, Newcastle, Piraeus and Singapore. As a full-service firm, they support clients at every stage of their business lifecycle – from start-up to multi-million-pound sales, and everything in between. The firm also offers specialized services in professional services and banking and financial law.' rating=None pricing=None service_tags=['Commercial Law', 'Banking and Financial Law', 'Professional Services Law'] provider_type='Company' country='UK' name='Hill Dickinson' logo=None website='https://www.hilldickinson.com' linkedin=None facebook=None instagram=None telephone=None mobile=None emails=None office_locations='Birmingham, Hong Kong, Leeds, Limassol, Liverpool, London, Manchester, Monaco, Newcastle, Piraeus, Singapore' key_individuals=Non

In [ ]:
# llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model).with_structured_output(ServiceProviderOutput)
# llm_chain = prompt_template | llm
# result = llm_chain.invoke({"html_content": ""})

/home/mohammed/anaconda3/envs/growbal/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [2]:
# Wrap synchronous ORM calls in sync_to_async
@sync_to_async
def get_or_create_user(email, username, password):
    user, created = CustomUser.objects.get_or_create(
        email=email,
        defaults={'username': username}
    )
    if created and password:
        user.set_password(password)
        user.save()
    return user.id

user_email = "jonathon.davidson@example.com"
user_username = "jonathon.davidson"
user_password = "securepassword123"

# If running in Jupyter or an async context, call as follows:
user_id = await get_or_create_user(user_email, user_username, user_password)

print(f"User ID: {user_id}")

User ID: 3


In [3]:
from services.serializers import ServiceSerializer
from accounts.serializers import ServiceProviderProfileSerializer
from asgiref.sync import sync_to_async

# service_data = {
#     "service_description": "Professional Legal Consulting",
#     "service_tags": ["legal", "consulting", "business"],
#     "ratings": "4.7/5",
#     "pricing": "Hourly rate starting at $150"
# }

# serializer = ServiceSerializer(data=service_data)

# if serializer.is_valid():
#     service = serializer.save()
#     print(f"Service '{service.service_description}' successfully created with tags: {service.service_tags}")
#     print(f"Service ID: {service.id}")
# else:
#     print("Failed to create service:")
#     print(serializer.errors)
    
service_data = {
    "service_description": "Professional Legal Consulting",
    "service_tags": ["legal", "consulting", "business"],
    "ratings": "4.7/5",
    "pricing": "Hourly rate starting at $150"
}

@sync_to_async
def create_service(data):
    serializer = ServiceSerializer(data=data)
    if serializer.is_valid():
        service = serializer.save()
        return {
            "id": service.id,
            "description": service.service_description,
            "tags": list(service.service_tags.names())
        }
    else:
        raise ValueError(serializer.errors)

# Direct async call (works in Jupyter/IPython):
result = await create_service(service_data)
print(f"Created service with ID: {result['id']}")
print(f"Description: {result['description']}")
print(f"Tags: {result['tags']}")

Created service with ID: 2
Description: Professional Legal Consulting
Tags: ['legal', 'consulting', 'business']


In [4]:
data = {
    "user": "ian@navs.com",
    "service": 1,  # existing Service ID or None
    "provider_type": "Company",
    "country": "USA",
    "session_status": "active",
    "tier": "Level 1",
    "name": "Doe Industries",
    "website": "https://doeindustries.com",
    "linkedin": "https://linkedin.com/company/doeindustries",
    "facebook": "",
    "instagram": "",
    "telephone": "+1 123-456-7890",
    "mobile": "+1 098-765-4321",
    "emails": "info@doeindustries.com",
    "office_locations": "New York, Los Angeles",
    "key_individuals": "John Doe, Jane Smith"
}

@sync_to_async
def create_service_provider_profile(data):
    serializer = ServiceProviderProfileSerializer(data=data)
    if serializer.is_valid():
        profile = serializer.save()
        return {
            "username": profile.user.username,
            "profile_id": profile.id,
            "name": profile.name,
        }
    else:
        print(serializer.errors)
        # raise ValueError(serializer.errors)

# Direct async call (Jupyter/IPython environment):
result = await create_service_provider_profile(data)
print(f"Profile created for user: {result['username']}")
print(f"Profile ID: {result['profile_id']}")
print(f"Company Name: {result['name']}")

IntegrityError: duplicate key value violates unique constraint "accounts_serviceproviderprofile_user_id_key"
DETAIL:  Key (user_id)=(4) already exists.


In [ ]:
# from accounts.models import ServiceProviderProfile
# from services.models import Service
# from accounts.serializers import ServiceProviderProfileSerializer
# from services.serializers import ServiceSerializer


In [ ]:
# print(result)

NameError: name 'result' is not defined